In [1]:
import os

In [2]:
%pwd

'c:\\Users\\admin\\Desktop\\project\\vinipro\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\admin\\Desktop\\project\\vinipro'

In [5]:
# dataclass ---> It imports a python tool called dataclass that helps you create classes 
# that only store data . It reduces the amount of code you need to write (no need to 
# manually write __init__ )  

# Pathlib ---> Imports the Path class , which helps you work with file and folder paths 
# easily and in a clean way for all operationg system .

# @dataclass(frozen=True)  ----> This is a decorator that tells python to dataclass means 
# we cannot change the values once the object is created . It makes the configuration 
# safe from accidental changes .

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



In [ ]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml , create_directories


In [13]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self):
        config = self.config.data_ingestion

        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )

        return data_ingestion_config

In [15]:
import os
import urllib.request as request
import zipfile
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

In [17]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename , headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

            logger.info(f"{filename} download! with following info: \n {headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path , exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file , 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [20]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


✅ YAML Content: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/adityasodani03/dlops_champions/raw/main/Chicken-fecal-images.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}
[2025-08-30 17:30:20,614: INFO: common: YAML file config\config.yaml loaded successfully]
✅ YAML Content: {'epochs': 1}
[2025-08-30 17:30:20,622: INFO: common: YAML file params.yaml loaded successfully]
[2025-08-30 17:30:20,630: INFO: common: created directory at: artifacts]
[2025-08-30 17:30:20,638: INFO: common: created directory at: artifacts/data_ingestion]
[2025-08-30 17:30:26,175: INFO: 3869023300: artifacts/data_ingestion/data.zip download! with following info: 
 Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec0126